## User Instructions
This is a program which will create R-Maps. These are used to correlate a voxel to a continuous outcome measure.
As it stands, this software employs Pearson Correlation Coefficients, which imply it will be best to have a continuous outcome on a percent scale. If you would like to do otherwise, a Spearman Correlation is possible. 

The software will walk you through everything. 

_____
# Nifti Configuration

**Files are expected to follow a BIDS naming convention.**

**Files are expected to have subject ID in them which is identical to subject ID in the CSV**

**Files are expected to be in 2x2x2 resolution**
_____
# CSV configuration:
**Subject IDs expected to be in the nifti names**

**Subject IDs expected to be in a column of your target CSV labelled "subject"**

Imports

In [1]:
import pandas as pd
import numpy as np

Save Information

-Enter the directory you would like to save to

In [3]:
out_dir = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/SANTE_DBS/derivatives/network_maps'

---

**Instructions**: Please fill out the `path` and `file_pattern` variables. 

The file_path is the shared base directory holding all files. ie) blah/blah/blah/BIDS

The file_pattern is the shared naming architectur in all files ie)  * / * / * subT1 * .nii

---

In [4]:
# What is the shared path to the folder/csv containing the nifti files/files paths for the neuroimaging files?
path_1 = '/data/nimlab/dl_archive/sante_vtas_2year_outcome_superconservative_mask'

#What is the shared file architecture of your neuroimaging files after the base path?
file_pattern = '*/connectivity/*stat-t*.nii.gz'

In [5]:
#-----------------DO NOT TOUCH--------------------------------------------------------
import os
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
df_1 = import_matrices_from_folder(path_1, file_pattern=file_pattern)
df_1

I will search:  /data/nimlab/dl_archive/sante_vtas_2year_outcome_superconservative_mask/*/connectivity/*stat-t*.nii.gz


""


**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID. For example in 04-mwp1glanat_resampled.nii, this is " "
2) Proceeding the subject ID. For example in 04-mwp1glanat_resampled.nii, this is "-mwp1"

In [6]:
from calvin_utils.file_utils.dataframe_utilities import extract_and_rename_subject_id

def preprocess_names(df, string_preceding_id, string_proceeding_id, cols=True):
    """
    Preprocess the given dataframe by extracting and renaming the subject ID, 
    then transposing the dataframe.

    Parameters:
    - df: The dataframe to preprocess.
    - string_preceding_id: String preceding the subject ID.
    - string_proceeding_id: String proceeding the subject ID.

    Returns:
    - The preprocessed dataframe.
    """
    split_command_dict = {string_preceding_id: 1, string_proceeding_id: 0}
    if cols:
        df = extract_and_rename_subject_id(dataframe=df, split_command_dict=split_command_dict).transpose()
    else:
        df = extract_and_rename_subject_id(dataframe=df, split_command_dict=split_command_dict)
    df.index.name = 'subject'
    return df


In [7]:
string_preceding_id = 'sub-'
string_proceeding_id = '_grey'

In [ ]:
df_1 = preprocess_names(df_1, string_preceding_id, string_proceeding_id)
df_1

Define the path to the CSV which has your clinical information

In [ ]:
path_2 = '/PHShome/cu135/github_repository/Research/nimlab/notebooks/sante_memoryScore_twoYearOutcome.csv'

In [ ]:
# Import a CSV with the clinical data of interest
df_2 = pd.read_csv(path_2)
df_2

In [ ]:
df_2 = df_2.loc[:, ['subject', 'Age', '% Change from baseline (ADAS-Cog11)']]

In [ ]:
subject_column = 'subject'

In [ ]:
string_preceding_id = ''
string_proceeding_id = ''

It is expected there is a columnc called which has subject information in it. The information in this column must correspond in the dataframe above. If it does not exist, add it to your CSV before proceeding. 

Define the column below using:

subject_colum = 

In [ ]:
import os
from calvin_utils.statistical_utils.voxelwise_statistical_testing import generate_r_map
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti

def reset_index_if_subject_is_index(df, subject_column):
    if subject_column in df.index.names:
        df.reset_index(inplace=True)
        # Reorder columns to make 'subject' the first column
        cols = [subject_column] + [col for col in df.columns if col != subject_column]
        df = df[cols]
    return df

def process_and_generate_maps(df_1, df_2, subject_column, out_dir, string_preceding_id, string_proceeding_id, mask_path=None, method='pearson'):
    """
    Process the given dataframes, and generate maps based on the columns.

    Parameters:
    - df_1: First dataframe.
    - df_2: Second dataframe.
    - subject_column: The column name referring to the subject.
    - out_dir: The output directory to save the generated maps.
    """
    # Check if 'subject' is in the index or columns for df_1
    if subject_column in df_1.index.names:
        df_1 = reset_index_if_subject_is_index(df_1, subject_column)
    if subject_column in df_2.index.names:
        df_2 = reset_index_if_subject_is_index(df_2, subject_column)
        
    df_1[subject_column] = df_1[subject_column].astype(str)
    if all(df_2[subject_column].apply(lambda x: type(x) is str)):
        if string_proceeding_id != '':
            df_2[subject_column] = [name.split(string_proceeding_id)[0] for name in df_2[subject_column]]
        if string_preceding_id != '':
            df_2[subject_column] = [name.split(string_preceding_id)[1] for name in df_2[subject_column]]
        print('extracting subject ID')
    else:
        df_2[subject_column] = df_2[subject_column].astype(str)

    # Iterate over column, avoiding the one with subject id in it
    for colname in [col for col in df_2.columns if col != subject_column]:
        print(f'Working on {colname}')
        merged_df = df_2[[colname, subject_column]].merge(df_1, on=subject_column, how='inner').set_index(subject_column)
        
        # Remove any rows with NaN values
        copy_df = merged_df.copy()
        try:
            merged_df.dropna(inplace=True)
            r_df, p_df, r_squared_df = generate_r_map(merged_df, mask_path=mask_path, method=method)

            view_and_save_nifti(p_df, os.path.join(out_dir, 'p_map', colname))
            view_and_save_nifti(r_df, os.path.join(out_dir, 'r_map', colname))
            view_and_save_nifti(r_squared_df, os.path.join(out_dir, 'r_squared_map', colname))
        except Exception as e:
            if "x and y must have length at least 2" in str(e):
                print('Caught exception: NaNs or Infs suspected in input data. Trying workaround.')
                copy_df.replace([np.inf, -np.inf], np.nan, inplace=True)
                copy_df.fillna(0, inplace=True)
                r_df, p_df, r_squared_df = generate_r_map(copy_df, mask_path=mask_path)

                view_and_save_nifti(p_df, os.path.join(out_dir, 'p_map', colname))
                view_and_save_nifti(r_df, os.path.join(out_dir, 'r_map', colname))
                view_and_save_nifti(r_squared_df, os.path.join(out_dir, 'r_squared_map', colname))
            else:
                print(f'Error {e}')
    return merged_df


What is the name of the column that contains your subject labels?

In [ ]:
import os
mask_path = None
merged_df = process_and_generate_maps(df_1.copy(), df_2.copy(), subject_column, out_dir, string_preceding_id, string_proceeding_id, mask_path=mask_path, method='spearman')

Your R-Maps have all been generated. Consider adding Calvin as a collaborator if this was useful!

-- Calvin

## Optional - Perform Delta R-Map and Permute it for Significance

**Calculate the Observed Delta-R Map Between 2 Populations**

In [ ]:
from calvin_utils.statistical_utils.voxelwise_statistical_testing import generate_delta_r_map
delta_matrix = merged_df.copy()
observed_delta_r_map = generate_delta_r_map(delta_matrix, threshold_of_interest=65, column_of_interest='Age at DOS')

In [ ]:
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
view_and_save_nifti(observed_delta_r_map, (out_dir+'/over_vs_under_65_delta_r_map'))

## Calculate the Empiric Delta-R Map Distribution 
### Note, this permutes the label of the population without permuting the neuroimaging data.
### Therefore, we are testing if the separation of the r-maps is significantly due to the variable of interest. 

In [ ]:
from calvin_utils.statistical_utils.voxelwise_statistical_testing import permuted_patient_label_delta_r_map
from calvin_utils.file_utils.print_suppression import HiddenPrints
n_permutations = 2
column_of_interest = 'Age at DOS'
threshold_of_interest = 65
with HiddenPrints():
    p_count_df = permuted_patient_label_delta_r_map(dataframe_to_permute=merged_df, 
                                                observed_delta_r_map=observed_delta_r_map, 
                                                column_of_interest=column_of_interest, 
                                                threshold_of_interest=threshold_of_interest, 
                                                n_permutations=n_permutations)

In [ ]:
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
view_and_save_nifti(p_values_df, (out_dir+'/over_vs_under_65_delta_r_map_p_values_df'))

Enjoy

--Calvin